# Example Model Metamer Generation from Audio Networks

This notebook walks through model metamer generation for an example sound as a demonstration of the optimization used in the paper: 

*Metamers of neural networks reveal divergence from human perceptual systems. Feather, J., Durango, A., Gonzalez, R., & McDermott, J. (2019). In Advances in Neural Information Processing Systems.*

(Tested in tensorflow 1.13, with no guarantee to work with other versions)

# Download tfcochleagram and pycochleagram
To generate audio metamers you will need tfcochleagram to build a tensorflow graph for the cochleagram generation (located in a separate github repo) which will also require pycochleagram to generate the cochlear filters:

https://github.com/jenellefeather/tfcochleagram

https://github.com/mcdermottLab/pycochleagram


# Download the model checkpoint, null distribution, and configuration pickle

### Before running this notebook, download the following file from the McDermott lab website

http://mcdermottlab.mit.edu/jfeather/model_metamers/assets/metamers_audio_models_network_files.tar.gz

(Warning: This file is ~2GB in size)

Untar the above file (`tar -xvf metamers_audio_models_network_files.tar.gz`). 

This file should contain the following: 

(1) Network configuration files used by the `build_*.py` scripts (`word_network_aliased.pckl`, `word_reduced_aliasing_null_dist_spearman_r.pckl`)

(2) Saved tensorflow checkpoints for both models

(3) Pre-computed null distributions for both models (`word_aliased_null_dist_spearman_r.pckl`, `word_reduced_aliasing_null_dist_spearman_r.pckl`)


# Description of included audio networks
Functions for two audio networks are included, the "Word Trained CNN" presented in Figure 3 and the reduced aliasing version of this network. A direct comparison of metamers from these two networks are in the left most plot of Figure 4, as shown below:

<img src="assets/audio_networks_figure_4.png" alt="Drawing" style="width: 400px;"/>

The below notebook is set up to generate metamers from the reduced aliasing network, and comments are included where the aliased network could be swapped in. 



# Now load the dependencies

In [1]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pylab as plt
import scipy
import sys
import os
import metamer_helpers
from lossfunctions import generate_loss_functions
import pickle
import IPython.display as ipd


# Set up the input to the graph

Loading the network should include any preprocessing that was performed on the input. For the audio networks, this includes the cochleagram generation graph built with tfcochleagram. 

Including the preprocessing in the metamer generation is important to ensure that generated metamers go through the same preprocessing that is applied to the natural input. 

All of this is included in the `build_word_network_aliased.py` or `build_word_network_reduced_aliasing.py` script. When generating metamers for a new network, a separate build script should be written including the preprocessing.

The build script also provides easy pointers to activation layers in the network, and applies the modified gradient ReLU to the desired layers. 


In [ ]:
tf.reset_default_graph()

# To load the reduced aliasing network
import build_word_network_reduced_aliasing as build_network
# To load the network with aliasing
# import build_word_network_aliased as build_network

nets, sess, metamer_layers = build_network.main()

# Make a function that re-initializes the input variable
input_tensor, input_noise_assign_op = metamer_helpers.make_initialization_op_pink_audio(
    nets['input_signal'], nets['input_signal'].get_shape().as_list()[1], 
    audio_scaling=0.1, rms_normalize=0.1)


# Load an example speech sound to generate metamers.

In [3]:
audio_path = 'assets/human_audio_resampled.wav'
wav_word = 'human'
audio_dict = metamer_helpers.use_audio_path_specified_audio(audio_path,
                                                            wav_word,
                                                            rms_normalize=0.1)


Loading: assets/human_audio_resampled.wav


# Set up a loss function. 

This is a partial function that we can apply to each of the metamer layers. For Feather et al. 2019 we use an L2 loss on the activations from single layers of the network. 

In [4]:
# The loss function used for metamer generation (L2 loss) is 'raw_pixels'
loss_function_name = 'raw_pixels' 
loss_function, measure_stats = generate_loss_functions(LOSS_TYPE=loss_function_name,
                                                       SHAPE_NORMALIZE=False)

# Run the optimization to generate metamers

Here we can run the optimization for each layer in `metamer_layers` and plot the example images. 

We optimize from one early layer of the network and one late layer of the network. 

Note: This will take a long time to run in a notebook especially for the late layers of the network.
For this demonstration, we chose two example layers to optimize.   

### Interpretation of Demos

By generating metamers from one early layer and one late layer as demonstrated below, it is clear that the late layer of the network no longer sounds like the matched sample of clean speech. This demonstrates that the model invariances are not the same as human invariances. 

In [5]:
# Uncomment to synthesize and plot all of the metamer layers (WARNING: SLOW)
# plot_metamer_layers = metamer_layers

# Uncomment the below to only run a few metamer layers for the reduced aliasing network
plot_metamer_layers = ['pool_0_0', 'conv_4_jittered_relu']

# Uncomment the below to only run a few metamer layers for the aliased network
# (The size of the activations at `pool_0_0` is matched to the size at `conv_0_jittered_relu`)
# plot_metamer_layers = ['conv_0_jittered_relu', 'conv_4_jittered_relu']

# Metamers in Feather et al. 2019 ran for 15000 iterations of Adam as the loss began to flatten for many 
# model metamers at this point. The learning rate the iterations may need to be changed for some models. 
iterations_adam = 15000 
log_loss_every_num = 1000 # used to track the loss and intermediate examples
starting_learning_rate_adam = 0.001

num_layers = len(plot_metamer_layers)

plt.figure(figsize=(3*num_layers,12))

metamer_dict = {}

print('Original audio')
ipd.display(ipd.Audio(audio_dict['wav'], rate=audio_dict['SR']))

for layer_idx, layer in enumerate(plot_metamer_layers): 
    print('Generating metamer for Word Network (Reduced Aliasing) layer %s'%layer)
    
    # Reinitialize the input variable
    sess.run(input_noise_assign_op)
    
    # Get the loss on the layer features
    orig_features = sess.run(nets[layer], feed_dict = {input_tensor:[audio_dict['wav']]})
    losses = loss_function(nets[layer], orig_features, update_losses=[])
    
    # In case we had included multiple losses, sum over them
    loss = tf.reduce_sum(losses)

    # Build the optimizers and run the optimization
    total_loss, track_iters, track_audio = metamer_helpers.run_optimization(
        loss, sess, input_tensor, iterations_adam, log_loss_every_num, 
        starting_learning_rate_adam=starting_learning_rate_adam)
    
    # Evaluate some of the tensors to save and to plot
    orig_predictions = sess.run(nets['logits'], feed_dict = {input_tensor:[audio_dict['wav']]})[0]
    synth_predictions = sess.run(nets['logits'])[0]
    synth_audio = sess.run(input_tensor)
    synth_features = sess.run(nets[layer]).ravel()
    
    synth_coch = np.squeeze(sess.run(nets['visualization_input']))
    orig_coch = np.squeeze(sess.run(nets['visualization_input'], feed_dict = {input_tensor:[audio_dict['wav']]}))
                            
    metamer_dict[layer] = {}
    metamer_dict[layer]['track_audio'] = track_audio
    metamer_dict[layer]['total_loss'] = total_loss
    metamer_dict[layer]['orig_features'] = orig_features
    metamer_dict[layer]['synth_features'] = synth_features
    metamer_dict[layer]['synth_audio'] = synth_audio
    metamer_dict[layer]['orig_audio'] = [audio_dict['wav']]
    metamer_dict[layer]['synth_audio'] = synth_coch
    metamer_dict[layer]['orig_audio'] = orig_coch
    metamer_dict[layer]['orig_predictions'] = orig_predictions
    metamer_dict[layer]['synth_predictions'] = synth_predictions
    
    print('Audio for layer %s Model Metamer'%layer)
    ipd.display(ipd.Audio(synth_audio, rate=audio_dict['SR']))

    # Make some plots in the notebook 
    if layer_idx == 0:
        plt.subplot(4, num_layers, 1)
        plt.imshow(orig_coch, origin='lower', cmap='Blues')
        plt.title('Original Cochleagram')
    plt.subplot(4, num_layers, layer_idx+1+num_layers)
    plt.imshow(synth_coch, origin='lower', cmap='Blues')
    plt.title('%s'%layer)
    ax=plt.subplot(4, num_layers, layer_idx+1+num_layers*2)
    plt.scatter(orig_predictions,synth_predictions)
    plt.ylim(ymin=np.min(orig_predictions),ymax=np.max(orig_predictions))
    plt.xlim(xmin=np.min(orig_predictions),xmax=np.max(orig_predictions))
    plt.title('Spearman R Predictions: %f'%(scipy.stats.spearmanr(orig_predictions,synth_predictions)[0]))

    ax=plt.subplot(4, num_layers, layer_idx+1+num_layers*3)
    plt.scatter(orig_features,synth_features)
    plt.ylim(ymin=np.min(orig_features.ravel()),ymax=np.max(orig_features.ravel()))
    plt.xlim(xmin=np.min(orig_features.ravel()),xmax=np.max(orig_features.ravel()))
    plt.title('Spearman R Activations: %f'%(scipy.stats.spearmanr(orig_features.ravel(),synth_features.ravel())[0]))


Original audio


Generating metamer for Word Network (Reduced Aliasing) layer pool_0_0
Instructions for updating:
Use tf.cast instead.
[0/15000], loss=14478.796875
[1000/15000], loss=112.249001
[2000/15000], loss=55.910007
[3000/15000], loss=46.313457
[4000/15000], loss=38.061726
[5000/15000], loss=33.091061
[6000/15000], loss=28.306278
[7000/15000], loss=27.642132
[8000/15000], loss=23.974846
[9000/15000], loss=20.760422
[10000/15000], loss=19.958574
[11000/15000], loss=19.329670
[12000/15000], loss=16.014658
[13000/15000], loss=17.604591
[14000/15000], loss=13.805872
[15000/15000], loss=13.933086
Audio for layer pool_0_0 Model Metamer


/usr/local/lib/python3.5/dist-packages/matplotlib/axes/_base.py:3604: MatplotlibDeprecationWarning: 
The `ymin` argument was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use `bottom` instead.
  alternative='`bottom`', obj_type='argument')
/usr/local/lib/python3.5/dist-packages/matplotlib/axes/_base.py:3610: MatplotlibDeprecationWarning: 
The `ymax` argument was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use `top` instead.
  alternative='`top`', obj_type='argument')
/usr/local/lib/python3.5/dist-packages/matplotlib/axes/_base.py:3215: MatplotlibDeprecationWarning: 
The `xmin` argument was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use `left` instead.
  alternative='`left`', obj_type='argument')
/usr/local/lib/python3.5/dist-packages/matplotlib/axes/_base.py:3221: MatplotlibDeprecationWarning: 
The `xmax` argument was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use `right` instead.
  alternative='`right`', obj_type='argument')


Generating metamer for Word Network (Reduced Aliasing) layer conv_4_jittered_relu
[0/15000], loss=17961.226562
[1000/15000], loss=72.148697
[2000/15000], loss=34.094360
[3000/15000], loss=17.431175
[4000/15000], loss=10.548532
[5000/15000], loss=6.729074
[6000/15000], loss=4.632000
[7000/15000], loss=3.719457
[8000/15000], loss=2.466436
[9000/15000], loss=2.529994
[10000/15000], loss=1.382218
[11000/15000], loss=6.132504
[12000/15000], loss=2.010914
[13000/15000], loss=0.821835
[14000/15000], loss=0.580759
[15000/15000], loss=0.491349
Audio for layer conv_4_jittered_relu Model Metamer


# For each layer, check that the optimized metamer meets the optimization criteria. 

(1) The network must predict the same thing for the synthetic and the original

(2) The Spearman R between the synthetic and the original must fall outside of the null distribution constructed on 1,000,000 images pairs (saved as `word_aliased_null_dist_spearman_r.pckl` or `word_reduced_aliasing_null_dist_spearman_r.pckl`). This is especially important for randomly initialized networks, where at the late layers the activations are all very correlated. 


In [7]:
# To load the null distribution for the reduced aliasing word network
null_distribution_file = 'word_reduced_aliasing_null_dist_spearman_r.pckl'
# To load the null distirbution for the aliased word network
# null_distribution_file = 'word_aliased_null_dist_spearman_r.pckl'

null_distirbution_spearman_r = pickle.load(open(null_distribution_file, 'rb'))

for layer_idx, layer in enumerate(plot_metamer_layers): 
    # Make sure that the Spearman R falls outside of the null distribution constructed from random image pairs
    spearman_r_metamer = scipy.stats.spearmanr(metamer_dict[layer]['orig_features'].ravel(), metamer_dict[layer]['synth_features'].ravel())[0]
    null_assertion = 'Synthesized metamer for layer %s falls within the null distribution of sounds. Optimization did not succeed.'%layer
    assert np.max(null_distirbution_spearman_r[layer]) < spearman_r_metamer, null_assertion
    
    # Make sure that the predicted class is the same for the original and the synthetic metamer
    class_assertion = 'Synthesized metamer for layer %s is not predicted as the same class as the original sound. Optimization did not succeed.'%layer
    assert np.argmax(metamer_dict[layer]['orig_predictions']) == np.argmax(metamer_dict[layer]['synth_predictions']), class_assertion

print('Metamers for all layers passed the optimization criteria!')


Metamers for all layers passed the optimization criteria!
